<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression 

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [2]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2022-10-18 19:15:21--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2022-10-18 19:15:21--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca08ee05cbe6ef58bc79fecdf7c.dl.dropboxusercontent.com/cd/0/get/BvE2Baq4RdD8U_a6TaDsGH8rm5ihQ9LK9eJ2YXNurvxsYBXIADDKxZHFCtp-uozZuTBhIKzjvNPMEcI6ZhYjtJxVOm_oBUl-uAm7ctvctpQrBc52E_q6aR_6ouNvwmQFLFnYBAiUpWCqQyGT36q0hZtPWKCI13fblKHlsPUkBAsvjA/file?dl=1# [following]
--2022-10-18 19:15:22--  https://uca08ee05cbe6ef58bc79fecdf7c.dl.dropboxusercontent.com/cd/0/get/BvE2Baq4RdD8U_a6TaDsGH8rm5ihQ9LK9eJ2YXNurvxsYBXIADDKxZHFCtp-uozZuTBhIKz

In [3]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [4]:
import numpy as np
import pandas as pd

def msle(ys, ps):
    assert len(ys) == len(ps)
    #################################
    # TODO: Implement this function #
    #################################
    return np.mean(np.square(np.log(1+ys) - np.log(1+ps)))

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [5]:
data_train = pd.read_csv('mieszkania.csv')
X_train = data_train.drop(columns=['cena'])
y_train = data_train['cena'].to_numpy().reshape(-1,1)
data_test = pd.read_csv('mieszkania_test.csv')
X_test = data_test.drop(columns=['cena'])
y_test = data_test['cena'].to_numpy().reshape(-1,1)
X_train.head()

,m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny
0,104,mokotowo,2,2,1940,1
1,43,ochotowo,1,1,1970,1
2,128,grodziskowo,3,2,1916,1
3,112,mokotowo,3,2,1920,1
4,149,mokotowo,3,3,1977,0


In [6]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################

mean_price_train = np.mean(y_train)
mean_prediction = np.full_like(y_train, mean_price_train)

msle_test = msle(y_train, mean_prediction)
print(msle_test)

0.3915250389336738


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [7]:
#############################################
# TODO: Find this constant and compute msle #
#############################################

c = np.exp(np.sum(np.log(1+y_train))/len(y_train)) - 1

mean_prediction = np.full_like(y_train, c)
msle_constant = msle(y_train, mean_prediction)

print(msle_constant)

0.36488961221465716


Now, let's implement a standard linear regression model. 

In [8]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################

def mse_loss(ys, ps):
    assert len(ys) == len(ps)
    return np.mean(np.square(ys-ps))

lr = 2e-9 # step size
n_epochs = 20000 # number of passes over the training data

def predict(X, w):
    return X @ w

def evaluate(X, y, w):
    return mse_loss(y, predict(X, w))


X_train_vec = X_train.drop(columns=['dzielnica']).to_numpy()
X_train_vec = np.concatenate([X_train_vec, np.ones((X_train_vec.shape[0], 1))], axis=-1)

X_test_vec = X_test.drop(columns=['dzielnica']).to_numpy()
X_test_vec = np.concatenate([X_test_vec, np.ones((X_test_vec.shape[0], 1))], axis=-1)

n_features = X_train_vec.shape[1]
w = np.ones((n_features, 1))

for i in range(n_epochs):
    y_hat = predict(X_train_vec, w)
    error = y_hat - y_train
    grad_w = X_train_vec.T @ error

    w = w - lr * grad_w
    
    # loss = evaluate(X_train_vec, y_train, w)
    # print(f'Iter: {i:>3} Loss: {loss:8.8f}')

prediction = predict(X_train_vec, w)
msle_lr = msle(y_train, prediction)
print(msle_lr)
# prediction = predict(X_train_vec, w)
# msle_lr = msle(y_train, prediction)
# print(msle_lr)

0.0544979826154857


Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [14]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################

lr = 8e-4
n_epochs = 30000

X_train_vec = X_train.drop(columns=['dzielnica']).to_numpy()
X_train_vec = np.concatenate([X_train_vec, np.ones((X_train_vec.shape[0], 1))], axis=-1)

X_test_vec = X_test.drop(columns=['dzielnica']).to_numpy()
X_test_vec = np.concatenate([X_test_vec, np.ones((X_test_vec.shape[0], 1))], axis=-1)


def fit(X, y, epochs, lr, debug=False):
  n_features = X.shape[1]
  w = np.ones((n_features, 1))
  for i in range(n_epochs):
      y_hat = predict(X, w)
      error = (np.log1p(y_hat) - np.log1p(y))
      grad_w = X.T @ error
      w = w - lr * grad_w

      if debug:
        loss = msle(y, y_hat)
        print(f'Iter: {i:>3} Loss: {loss:8.8f}')
  
  return w
    
def eval(X, y_true, w):
  prediction = predict(X, w)
  msle_loss = msle(y_true, prediction)
  return msle_loss


w = fit(X_train_vec, y_train, n_epochs, lr)
msle_train = eval(X_train_vec, y_train, w)
msle_test = eval(X_test_vec, y_test, w)
print(f'Train loss: {msle_train}')
print(f'Test  loss: {msle_test}')

Train loss: 0.05302723978146017
Test  loss: 0.08054085102342762


$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [27]:
###############################################
# TODO: Implement the feature engineering part #
###############################################

from sklearn.preprocessing import StandardScaler


X_train_vec = X_train.drop(columns=['dzielnica', 'm2']).to_numpy()
districts_train = pd.get_dummies(X_train['dzielnica'], prefix='dzielnia')
districts_train = districts_train.apply(lambda col: col*X_train['m2'])
X_train_vec = np.concatenate([X_train_vec, districts_train], axis=-1)
# scaler = StandardScaler()
# X_train_vec = scaler.fit_transform(X_train_vec)

X_train_vec = np.concatenate([X_train_vec, np.ones((X_train_vec.shape[0], 1))], axis=-1)



X_test_vec = X_test.drop(columns=['dzielnica', 'm2']).to_numpy()
districts_test = pd.get_dummies(X_test['dzielnica'], prefix='dzielnia')
districts_test = districts_test.apply(lambda col: col*X_test['m2'])
districts_test.reindex(columns = districts_train.columns, fill_value=0)
X_test_vec = np.concatenate([X_test_vec, districts_test], axis=-1)
# X_test_vec = scaler.transform(X_test_vec)

X_test_vec = np.concatenate([X_test_vec, np.ones((X_test_vec.shape[0], 1))], axis=-1)

lr = 9e-4
n_epochs = 30000

w = fit(X_train_vec, y_train, n_epochs, lr)
print(w)

[[4.41244310e+02]
 [3.02078316e+02]
 [1.18755844e+01]
 [4.58685643e+01]
 [4.05557995e+03]
 [7.14906191e+03]
 [7.30043370e+03]
 [5.20985148e+03]
 [2.48108490e-02]]


In [28]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################

msle_train = eval(X_train_vec, y_train, w)
msle_test = eval(X_test_vec, y_test, w)
print(f'Train MSLE loss: {msle_train}')
print(f'Test MSLE  loss: {msle_test}')

Train MSLE loss: 0.007410406876304113
Test MSLE  loss: 0.01704737964822043
